In [33]:
import os
import pickle
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from utils import raw_data_path, feature_data_path,result_path,cache_pkl_path,dump_pickle,load_pickle
from smooth import BayesianSmoothing
from _2_4_gen_predict_category_property import add_category_predict_rank

# 单特征smooth平滑ctr。历史

In [34]:
def gen_features_smooth_ctr():
    '''
    贝叶斯平滑版
    提取每天前些天的，分别以feature=['user_id', 'item_id', 'item_brand_id', 'shop_id']分类的，总点击次数_I,总购买次数_C,点击率_CTR
    以['day', feature, I_alias, C_alias, CTR_alias]存储
    文件名，【】_CTR.pkl
    '''
    all_data = load_pickle(raw_data_path+'all_data.pkl')    
    all_data = add_category_predict_rank(all_data)
    for feature in tqdm(['user_id', 'category_predict_rank', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
                         'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level', 
                         'item_collected_level', 'item_pv_level',
                         'context_page_id', 
                         'shop_id', 'shop_review_num_level', 'shop_star_level', 'hour']):  
       
        feature_path = feature_data_path+feature+'_smooth_CTR.pkl' #要存放的目录
        if os.path.exists(feature_path):
            print('found ' + feature_path)
        else:
            
            alpha_beta_dispaly = []
            
            print('generating ' + feature_path)
            I_alias = feature+'_smooth_I' #总点击次数
            C_alias = feature+'_smooth_C' #购买次数
            CTR_alias = feature+'_smooth_CTR'
            history_ctr = pd.DataFrame()
            for day in range(19,26):
                
                history_data = all_data[all_data['day'] < day]
                I = history_data.groupby([feature]).size().reset_index().rename(columns={0: I_alias})
                C = history_data[history_data['is_trade'] == 1].groupby([feature]).size().reset_index().rename(columns={0: C_alias})
                CTR = pd.merge(I, C, how='left', on=[feature])
                CTR[C_alias] = CTR[C_alias].fillna(0)
                
                hyper = BayesianSmoothing(1, 1)
                hyper.update(CTR[I_alias].values, CTR[C_alias].values, 10000, 0.0000001)
                alpha = hyper.alpha
                beta = hyper.beta
                
                alpha_beta_dispaly.append(alpha)
                alpha_beta_dispaly.append(beta)
                
                print(feature)
                print(alpha_beta_dispaly)
                dump_pickle(alpha_beta_dispaly,feature_data_path+'1'+feature+'.pkl')  #存储
                
                CTR[CTR_alias] = (CTR[C_alias] + alpha) / (CTR[I_alias] + alpha + beta)
                CTR['day'] = day
                history_ctr = history_ctr.append(CTR)
            
            print('-----------------------------------------------------------------------')
            print(feature)
            print(alpha_beta_dispaly)
            dump_pickle(alpha_beta_dispaly,feature_data_path+'1'+feature+'.pkl')  #存储
            print('-----------------------------------------------------------------------')
            dump_pickle(history_ctr[['day', feature, I_alias, C_alias, CTR_alias]],feature_path)  #存储

def add_features_smooth_ctr(all_data):
    '''
    向总体数据添加特征
    feature=['user_id', 'item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label', 'item_price_level', 'category_predict_rank']
    拼接键[feature, 'day']
    '''
    for feature in tqdm(['user_id', 'category_predict_rank', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
                         'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level', 
                         'item_collected_level', 'item_pv_level',
                         'context_page_id', 
                         'shop_id', 'shop_review_num_level', 'shop_star_level', 'hour']):  
        feature_path = feature_data_path+feature+'_smooth_CTR.pkl'
        if not os.path.exists(feature_path):
            gen_features_smooth_ctr()
        ctr_data = load_pickle(feature_path)
        all_data = pd.merge(all_data, ctr_data, how='left', on=[feature, 'day'])
    return all_data       

# 单特征无平滑ctr前一天

In [35]:
def gen_features_day_ctr():
    '''
    贝叶斯平滑版
    提取每天前些天的，分别以feature=['user_id', 'item_id', 'item_brand_id', 'shop_id']分类的，总点击次数_I,总购买次数_C,点击率_CTR
    以['day', feature, I_alias, C_alias, CTR_alias]存储
    文件名，【】_CTR.pkl
    '''
    all_data = load_pickle(raw_data_path+'all_data.pkl')    
    all_data = add_category_predict_rank(all_data)
    for feature in tqdm(['user_id', 'category_predict_rank', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
                         'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level', 
                         'item_collected_level', 'item_pv_level',
                         'context_page_id', 
                         'shop_id', 'shop_review_num_level', 'shop_star_level',]):  
        feature_path = feature_data_path+feature+'_day_CTR.pkl' #要存放的目录
        if os.path.exists(feature_path):
            print('found ' + feature_path)
        else:
            
            alpha_beta_dispaly = []
            
            print('generating ' + feature_path)
            I_alias = feature+'_day_I' #总点击次数
            C_alias = feature+'_day_C' #购买次数
            CTR_alias = feature+'_CTR'
            history_ctr = pd.DataFrame()
            for day in range(19,26):
                
                history_data = all_data[all_data['day'] == day-1]
                I = history_data.groupby([feature]).size().reset_index().rename(columns={0: I_alias})
                C = history_data[history_data['is_trade'] == 1].groupby([feature]).size().reset_index().rename(columns={0: C_alias})
                CTR = pd.merge(I, C, how='left', on=[feature])
                CTR[C_alias] = CTR[C_alias].fillna(0)
                
                CTR[CTR_alias] = CTR[C_alias] / CTR[I_alias]
                CTR['day'] = day
                history_ctr = history_ctr.append(CTR)
            
#             print('-----------------------------------------------------------------------')
#             print(feature)
#             print(alpha_beta_dispaly)
#             dump_pickle(alpha_beta_dispaly,feature_data_path+'1_day_'+feature+'.pkl')  #存储
#             print('-----------------------------------------------------------------------')
            dump_pickle(history_ctr[['day', feature, I_alias, C_alias, CTR_alias]],feature_path)  #存储

def add_features_day_ctr(all_data):
    '''
    向总体数据添加特征
    feature=['user_id', 'item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label', 'item_price_level', 'category_predict_rank']
    拼接键[feature, 'day']
    '''
    for feature in tqdm(['user_id', 'category_predict_rank', 'user_occupation_id', 'user_age_level', 'user_gender_id', 'user_star_level',
                         'item_id', 'item_brand_id', 'item_city_id', 'category2_label','item_price_level','item_sales_level', 
                         'item_collected_level', 'item_pv_level',
                         'context_page_id', 
                         'shop_id', 'shop_review_num_level', 'shop_star_level',]):  
        feature_path = feature_data_path+feature+'_day_CTR.pkl'
        if not os.path.exists(feature_path):
            gen_features_day_ctr()
        ctr_data = load_pickle(feature_path)
        all_data = pd.merge(all_data, ctr_data, how='left', on=[feature, 'day'])
        
        all_data[feature+'_day_I'] = all_data[feature+'_day_I'].fillna(0)
        all_data[feature+'_day_C'] = all_data[feature+'_day_C'].fillna(0)
    return all_data       

# user_id前一天点击某某某的数量

In [36]:
def gen_features_cross_day_ctr():
    '''
    贝叶斯平滑版
    提取每天前些天的，分别以feature=['user_id', 'item_id', 'item_brand_id', 'shop_id']分类的，总点击次数_I,总购买次数_C,点击率_CTR
    以['day', feature, I_alias, C_alias, CTR_alias]存储
    文件名，【】_CTR.pkl
    '''
    all_data = load_pickle(raw_data_path+'all_data.pkl')    
 
    for feature in tqdm(['user_id']):
   
        for feature2 in tqdm(['item_id', 'item_brand_id', 'shop_id', 'category2_label', 'item_price_level', ]):  
            
            feature_path = feature_data_path+feature+'_'+feature2+'_before_day_CTR.pkl' #要存放的目录
            if os.path.exists(feature_path):
                print('found ' + feature_path)
            else:

                print('generating ' + feature_path)
                I_alias = feature+'_'+feature2+'_day_I' #总点击次数
                C_alias = feature+'_'+feature2+'_day_C' #购买次数
             
                history_ctr = pd.DataFrame()
                for day in range(19,26):

                    history_data = all_data[all_data['day'] == day-1]
                    I = history_data.groupby([feature, feature2]).size().reset_index().rename(columns={0: I_alias})
                    C = history_data[history_data['is_trade'] == 1].groupby([feature, feature2]).size().reset_index().rename(columns={0: C_alias})
                    CTR = pd.merge(I, C, how='left', on=[feature, feature2])
                    CTR[C_alias] = CTR[C_alias].fillna(0)
                    CTR['day'] = day
                    history_ctr = history_ctr.append(CTR)

                dump_pickle(history_ctr[['day', feature, feature2, I_alias, C_alias]],feature_path)  #存储

def add_features_cross_day_ctr(all_data):
    '''
    向总体数据添加特征
    feature=['user_id', 'item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label', 'item_price_level', 'category_predict_rank']
    拼接键[feature, 'day']
    '''
    for feature in tqdm(['user_id',]):
   
        for feature2 in tqdm(['item_id', 'item_brand_id', 'shop_id', 'category2_label', 'item_price_level',]):  
            
            I_alias = feature+'_'+feature2+'_day_I' #总点击次数
            C_alias = feature+'_'+feature2+'_day_C' #购买次数
            feature_path = feature_data_path+feature+'_'+feature2+'_before_day_CTR.pkl' #要存放的目录
       
            if not os.path.exists(feature_path):
                gen_features_cross_day_ctr()
            ctr_data = load_pickle(feature_path)
            all_data = pd.merge(all_data, ctr_data, how='left', on=[feature, feature2, 'day'])
            all_data[I_alias] = all_data[I_alias].fillna(0)
            all_data[C_alias] = all_data[C_alias].fillna(0)
    return all_data       

# user_id历史点击某某某的数量

In [37]:
def gen_features_cross_history_ctr():
    '''
    贝叶斯平滑版
    提取每天前些天的，分别以feature=['user_id', 'item_id', 'item_brand_id', 'shop_id']分类的，总点击次数_I,总购买次数_C,点击率_CTR
    以['day', feature, I_alias, C_alias, CTR_alias]存储
    文件名，【】_CTR.pkl
    '''
    all_data = load_pickle(raw_data_path+'all_data.pkl')    
 
    for feature in tqdm(['user_id']):
   
        for feature2 in tqdm(['item_id', 'item_brand_id', 'shop_id', 'category2_label', 'item_price_level',]):  
            
            feature_path = feature_data_path+feature+'_'+feature2+'_before_history_CTR.pkl' #要存放的目录
            if os.path.exists(feature_path):
                print('found ' + feature_path)
            else:

                print('generating ' + feature_path)
                I_alias = feature+'_'+feature2+'_history_I' #总点击次数
                C_alias = feature+'_'+feature2+'_history_C' #购买次数
             
                history_ctr = pd.DataFrame()
                for day in range(19,26):

                    history_data = all_data[all_data['day'] <= day-1]
                    I = history_data.groupby([feature, feature2]).size().reset_index().rename(columns={0: I_alias})
                    C = history_data[history_data['is_trade'] == 1].groupby([feature, feature2]).size().reset_index().rename(columns={0: C_alias})
                    CTR = pd.merge(I, C, how='left', on=[feature, feature2])
                    CTR[C_alias] = CTR[C_alias].fillna(0)
                    CTR['day'] = day
                    history_ctr = history_ctr.append(CTR)

                dump_pickle(history_ctr[['day', feature, feature2, I_alias, C_alias]],feature_path)  #存储

def add_features_cross_history_ctr(all_data):
    '''
    向总体数据添加特征
    feature=['user_id', 'item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label', 'item_price_level', 'category_predict_rank']
    拼接键[feature, 'day']
    '''
    for feature in tqdm(['user_id',]):
   
        for feature2 in tqdm(['item_id', 'item_brand_id', 'shop_id', 'category2_label', 'item_price_level',]):  
            
            I_alias = feature+'_'+feature2+'_history_I' #总点击次数
            C_alias = feature+'_'+feature2+'_history_C' #购买次数
            feature_path = feature_data_path+feature+'_'+feature2+'_before_history_CTR.pkl' #要存放的目录
       
            if not os.path.exists(feature_path):
                gen_features_cross_history_ctr()
            ctr_data = load_pickle(feature_path)
            all_data = pd.merge(all_data, ctr_data, how='left', on=[feature, feature2, 'day'])
            all_data[I_alias] = all_data[I_alias].fillna(0)
            all_data[C_alias] = all_data[C_alias].fillna(0)
    return all_data       

# 特征交叉，有平滑，历史

In [ ]:
def gen_features_cross_smooth_ctr():
    '''
    贝叶斯平滑版
    提取每天前些天的，分别以feature=['user_id', 'item_id', 'item_brand_id', 'shop_id']分类的，总点击次数_I,总购买次数_C,点击率_CTR
    以['day', feature, I_alias, C_alias, CTR_alias]存储
    文件名，【】_CTR.pkl
    '''
    all_data = load_pickle(raw_data_path+'all_data.pkl')    
 
    for feature in tqdm(['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']):
   
        for feature2 in tqdm(['item_id', 'item_brand_id', 'shop_id', 'item_price_level', 'hour']):  
            
            feature_path = feature_data_path+feature+'_'+feature2+'_smooth_CTR.pkl' #要存放的目录
            if os.path.exists(feature_path):
                print('found ' + feature_path)
            else:

                alpha_beta_dispaly = []

                print('generating ' + feature_path)
                I_alias = feature+'_'+feature2+'_smooth_I' #总点击次数
                C_alias = feature+'_'+feature2+'_smooth_C' #购买次数
                CTR_alias = feature+'_'+feature2+'_smooth_CTR'
                history_ctr = pd.DataFrame()
                for day in range(19,26):

                    history_data = all_data[all_data['day'] < day]
                    I = history_data.groupby([feature, feature2]).size().reset_index().rename(columns={0: I_alias})
                    C = history_data[history_data['is_trade'] == 1].groupby([feature, feature2]).size().reset_index().rename(columns={0: C_alias})
                    CTR = pd.merge(I, C, how='left', on=[feature, feature2])
                    CTR[C_alias] = CTR[C_alias].fillna(0)

                    hyper = BayesianSmoothing(1, 1)
                    hyper.update(CTR[I_alias].values, CTR[C_alias].values, 1000, 0.000001)
                    alpha = hyper.alpha
                    beta = hyper.beta

                    alpha_beta_dispaly.append(alpha)
                    alpha_beta_dispaly.append(beta)

                    print(feature)
                    print(alpha_beta_dispaly)
                    dump_pickle(alpha_beta_dispaly,feature_data_path+'1'+feature+'_'+feature2+'.pkl')  #存储

                    CTR[CTR_alias] = (CTR[C_alias] + alpha) / (CTR[I_alias] + alpha + beta)
                    CTR['day'] = day
                    history_ctr = history_ctr.append(CTR)

                print('-----------------------------------------------------------------------')
                print(feature)
                print(alpha_beta_dispaly)
                dump_pickle(alpha_beta_dispaly,feature_data_path+'1'+feature+'_'+feature2+'.pkl')  #存储
                print('-----------------------------------------------------------------------')
                dump_pickle(history_ctr[['day', feature, feature2, I_alias, C_alias, CTR_alias]],feature_path)  #存储

def add_features_cross_smooth_ctr(all_data):
    '''
    向总体数据添加特征
    feature=['user_id', 'item_id', 'item_brand_id', 'shop_id', 'context_page_id', 'category2_label', 'item_price_level', 'category_predict_rank']
    拼接键[feature, 'day']
    '''
    for feature in tqdm(['user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']):
   
        for feature2 in tqdm(['item_id', 'item_brand_id', 'shop_id', 'item_price_level', 'hour']):  
            
            feature_path = feature_data_path+feature+'_'+feature2+'_smooth_CTR.pkl' #要存放的目录
       
            if not os.path.exists(feature_path):
                gen_features_cross_smooth_ctr()
            ctr_data = load_pickle(feature_path)
            all_data = pd.merge(all_data, ctr_data, how='left', on=[feature, feature2, 'day'])
    return all_data     

# 测试

In [ ]:
if __name__ =='__main__':
    all_data = load_pickle(raw_data_path+'all_data.pkl')
    
    all_data = add_category_predict_rank(all_data)
    all_data = add_features_smooth_ctr(all_data)
    all_data = add_features_day_ctr(all_data)
    all_data = add_features_cross_day_ctr(all_data)
    all_data = add_features_cross_smooth_ctr(all_data)
    all_data = add_features_cross_history_ctr(all_data)
    
    print(all_data.columns)  

generating ../features/category_predict_rank.pkl


  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

found ../features/user_id_smooth_CTR.pkl
generating ../features/category_predict_rank_smooth_CTR.pkl




  6%|▌         | 579/10000 [00:00<00:01, 5780.76it/s]

  9%|▉         | 923/10000 [00:00<00:01, 4798.08it/s]

 13%|█▎        | 1251/10000 [00:00<00:02, 4210.70it/s]

 15%|█▌        | 1547/10000 [00:00<00:02, 3734.80it/s]

 21%|██▏       | 2149/10000 [00:00<00:01, 4214.31it/s]

 28%|██▊       | 2751/10000 [00:00<00:01, 4630.10it/s]

 34%|███▎      | 3350/10000 [00:00<00:01, 4968.20it/s]

 40%|███▉      | 3954/10000 [00:00<00:01, 5245.29it/s]

 46%|████▌     | 4561/10000 [00:00<00:00, 5467.08it/s]

 52%|█████▏    | 5169/10000 [00:01<00:00, 5637.19it/s]

 58%|█████▊    | 5761/10000 [00:01<00:00, 5718.59it/s]

 64%|██████▎   | 6356/10000 [00:01<00:00, 5784.07it/s]

 70%|██████▉   | 6964/10000 [00:01<00:00, 5868.06it/s]

 76%|███████▌  | 7572/10000 [00:01<00:00, 5928.80it/s]

 82%|████████▏ | 8178/10000 [00:01<00:00, 5967.25it/s]

 88%|████████▊ | 8795/10000 [00:01<00:00, 6024.36it/s]

 94%|█████████▍| 9399/10000 [00:01<00:00, 6020.71it/s]

100%|██████████| 10000/10000 [00:01<00:00, 5553.

category_predict_rank
[13.990939137857882, 843.30633748243463]


...





  3%|▎         | 27/1000 [00:00<00:09, 107.06it/s]



  5%|▌         | 50/1000 [00:00<00:07, 126.81it/s]



  7%|▋         | 73/1000 [00:00<00:06, 146.09it/s]



 10%|▉         | 96/1000 [00:00<00:05, 163.31it/s]



 12%|█▏        | 119/1000 [00:00<00:04, 178.12it/s]



 14%|█▍        | 142/1000 [00:00<00:04, 189.59it/s]



 16%|█▋        | 165/1000 [00:00<00:04, 199.64it/s]



 19%|█▉        | 188/1000 [00:00<00:03, 206.42it/s]



 21%|██        | 211/1000 [00:01<00:03, 211.42it/s]



 23%|██▎       | 234/1000 [00:01<00:03, 215.40it/s]



 26%|██▌       | 257/1000 [00:01<00:03, 217.19it/s]



 28%|██▊       | 280/1000 [00:01<00:03, 219.86it/s]



 30%|███       | 303/1000 [00:01<00:03, 219.85it/s]



 33%|███▎      | 326/1000 [00:01<00:03, 220.66it/s]



 35%|███▍      | 349/1000 [00:01<00:02, 220.92it/s]



 37%|███▋      | 372/1000 [00:01<00:03, 188.44it/s]



 39%|███▉      | 392/1000 [00:01<00:03, 154.45it/s]



 41%|████      | 410/1000 [00:02<00:04, 137.62it/s]



 43%|████▎

user_star_level
[2.9661838048620663, 133.6486049436111, 2.8336339304242339, 125.2407154931465, 2.6282774762894943, 116.31700505633025, 2.5837754741639776, 113.33923576473678, 2.4617420580186766, 107.7473568093635, 2.4131772301894125, 106.52238146470255]






  1%|          | 10/1000 [00:00<00:10, 91.03it/s]



  3%|▎         | 26/1000 [00:00<00:09, 104.15it/s]



  5%|▍         | 49/1000 [00:00<00:07, 123.99it/s]



  7%|▋         | 72/1000 [00:00<00:06, 142.95it/s]



 10%|▉         | 95/1000 [00:00<00:05, 159.83it/s]



 12%|█▏        | 118/1000 [00:00<00:05, 174.49it/s]



 14%|█▍        | 140/1000 [00:00<00:04, 185.30it/s]



 16%|█▌        | 162/1000 [00:00<00:04, 193.56it/s]



 18%|█▊        | 184/1000 [00:00<00:04, 199.23it/s]



 21%|██        | 206/1000 [00:01<00:03, 204.55it/s]



 23%|██▎       | 229/1000 [00:01<00:03, 209.44it/s]



 25%|██▌       | 252/1000 [00:01<00:03, 213.04it/s]



 27%|██▋       | 274/1000 [00:01<00:03, 212.92it/s]



 30%|██▉       | 297/1000 [00:01<00:03, 215.02it/s]



 32%|███▏      | 319/1000 [00:01<00:03, 215.05it/s]



 34%|███▍      | 341/1000 [00:01<00:03, 212.89it/s]



 36%|███▋      | 363/1000 [00:01<00:03, 211.49it/s]



 38%|███▊      | 385/1000 [00:01<00:03, 160.39it/s]



 40%|████   

user_star_level
[2.9661838048620663, 133.6486049436111, 2.8336339304242339, 125.2407154931465, 2.6282774762894943, 116.31700505633025, 2.5837754741639776, 113.33923576473678, 2.4617420580186766, 107.7473568093635, 2.4131772301894125, 106.52238146470255, 2.3683062047159971, 104.50925592788919]
-----------------------------------------------------------------------
user_star_level
[2.9661838048620663, 133.6486049436111, 2.8336339304242339, 125.2407154931465, 2.6282774762894943, 116.31700505633025, 2.5837754741639776, 113.33923576473678, 2.4617420580186766, 107.7473568093635, 2.4131772301894125, 106.52238146470255, 2.3683062047159971, 104.50925592788919]
-----------------------------------------------------------------------



 60%|██████    | 3/5 [00:01<00:00,  2.51it/s]

In [ ]:
for c in all_data.columns:
    print(c)